In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
dataset='sst2'
model_name='meta-llama/Llama-2-7b-hf'

results_path = f'./checkpoints/{dataset}/{model_name}/textfooler/'

attacks = ['textbugger', 'textfooler']
ks = [1, 3, 7, 15, 31, 63, 127, 255, 511]
results = []
for seed in [1]:
    for shot in [128, 256, 512]:
        for k in [x for x in ks if x < shot]:
            for attack in attacks:
                try:
                    file = f'{results_path}knn_icl-seed-{seed}-shot-{shot}/example-k-{k}/logs_{attack}_test.txt'
                    """
                    locate and parse the following in the file 
                    +-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 534    |
| Number of failed attacks:     | 178    |
| Number of skipped attacks:    | 160    |
| Original accuracy:            | 81.65% |
| Accuracy under attack:        | 20.41% |
| Attack success rate:          | 75.0%  |
| Average perturbed word %:     | 8.36%  |
| Average num. words per input: | 17.4   |
| Avg num queries:              | 74.37  |
| Adv confidence:               | 1.0    |
| Average Original Perplexity:  | 61.62  |
| Average Attack Perplexity:    | 86.19  |
| Average Attack USE Score:     | 0.88   |
+-------------------------------+--------+
                    """ 
                    with open(file, 'r') as f:
                        lines = f.readlines()
                except:
                    print(f'file {file} not found')
                    continue

                result = {'seed': seed, 'shot': shot, 'k': k, 'attack': attack}
                for line in lines:
                    if 'Original accuracy:' in line:
                        result['Clean Acc'] = float(line.split(':')[1].strip().replace('%', '').strip('| '))
                    elif 'Accuracy under attack:' in line:
                        result['Attack Acc'] = float(line.split(':')[1].strip().replace('%', '').strip('| '))
                    elif 'Attack success rate:' in line:
                        result['ASR'] = float(line.split(':')[1].strip().replace('%', '').strip('| '))

                results.append(result)


file ./checkpoints/sst2/meta-llama/Llama-2-7b-hf/textfooler/knn_icl-seed-1-shot-512/example-k-511/logs_textbugger_test.txt not found


In [9]:
pd.DataFrame(results)

,seed,shot,k,attack,Clean Acc,Attack Acc,ASR
0,1,128,1,textbugger,81.65,32.00,60.81
1,1,128,1,textfooler,81.65,20.41,75.00
2,1,128,3,textbugger,85.78,17.43,79.68
3,1,128,3,textfooler,85.78,8.26,90.37
4,1,128,7,textbugger,87.39,19.15,78.08
5,1,128,7,textfooler,87.39,7.80,91.08
6,1,128,15,textbugger,87.84,22.25,74.67
7,1,128,15,textfooler,87.84,10.32,88.25
8,1,128,31,textbugger,88.65,21.22,76.07
9,1,128,31,textfooler,88.65,9.63,89.13
